In [ ]:
#!pip install redis
#!pip install mysql-connector-python 
#!pip install pymongo

## Importing Libraries

In [1]:
import json
import tweepy
import sys
import mysql.connector 
import pymongo
import re
import time
from datetime import datetime
import redis
import fontstyle
from IPython.display import clear_output

## Streaming data from Twitter API

In [ ]:
ckey ='e3OLPfNMEIsBXc22Rw8TpmXaw'
csec ='iofJJbpiC9nAPnFHpKpfGh7yVRUXlbXQuVxRhzRiLd2cqSIXTl'
akey ='1363132021053227013-CXxuCEIzH3foO7AY8DWPnLEfi0Zzdp'
asec ='kqnwucwfUeVmlioUvTJcCR0Hvocie4nVTyzDLTQPUfggk'

In [ ]:
auth = tweepy.OAuthHandler(ckey, csec)
auth.set_access_token(akey, asec)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

In [ ]:
dummy = 0
tweet_max = 11000

In [ ]:
class StdOutListener(tweepy.StreamListener):
    dummy = 0
    tweet_max = 11000
    def on_data(self, data):
        if self.dummy<self.tweet_max:
            if self.dummy==0:
                print("[")
            print (data)
            if self.dummy!=(self.tweet_max-1):
                print(",")
            self.dummy+=1
            return True
        else:
            print("]")
            return False
        
    def on_error(self, status):
        print (status)

In [ ]:
###Printed output directly to the JSON file 
default_stdout = sys.stdout

sys.stdout = open('04.25.2020 tweets_5.json', 'w')
tweets_listener = StdOutListener()
stream = tweepy.Stream(api.auth, tweets_listener)
stream.filter(track=["#doge","#dogecoin","#bitcoin","#BTC"], languages=["en"])

sys.stdout = default_stdout

In [3]:
tweets_chk = open(r'04.25.2020 tweets_5.json',"r")
tweets_json = json.load(tweets_chk)

## Instantiating SQL DB

In [39]:
userdb = mysql.connector.connect(host="localhost",user="root",passwd="raju437@.", database ="user")

if(userdb):
    print("Connection success")
else:
    print("Connection failed")
    
cursor = userdb.cursor(buffered = True)


cursor.execute("CREATE TABLE user (\
               user_id       CHAR(255),\
               user_name     CHAR(255),\
               handle        CHAR(255),\
               description   VARCHAR(500),\
              followers     BIGINT,\
               statuses      INT,\
               created_date  DATETIME,\
               verified      CHAR(5),\
               friends       INT,\
               favourites    INT,\
               protected     CHAR(50),\
               INDEX (user_name) )")  
#cursor.execute('drop table user')

Connection success


## Instantiating MongoDB collection

In [5]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
tweets_db = client["tweet_db"]
#tweets_col.drop()
tweets_col = tweets_db["tweets_col"]

## Cleaning data and Pushing data into MySQL and MongoDB

In [ ]:
for record in tweets_json:
    
    ###Starting with SQL insertion
    user = record['user']
    inp = [user['id_str'],user['name'], user['screen_name'],user['description'], user['followers_count'], 
                  user['statuses_count'], user['verified'],user['created_at'],user['friends_count'],user['favourites_count'],user['protected']]
    
    #Cleaning commas from User name and Description 
    inp[1] =  re.sub("'","",inp[1])
    if inp[3] is not None:
        inp[3] =  re.sub("'","",inp[3])
        
    #Converting tweepy created_date to datetime format     
    inp[7] = datetime.strftime(datetime.strptime(inp[7],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
    
    
    #Updating the user information for the users who are already present in the user data base
    cursor.execute("select user_id from user where user_id ='{}'".format(inp[0]))

    result = cursor.fetchone()
    if result:
        query = "UPDATE user SET description = '{}',followers= {},statuses= {},verified= '{}',friends={},favourites ={},protected='{}' WHERE user_id  = '{}'".format(inp[3],inp[4],inp[5],inp[6],inp[8],inp[9],inp[10],inp[0])
    else:
        query = ("INSERT INTO user (user_id, user_name, handle,description,followers,statuses,verified,created_date,friends,favourites,protected)" 
             "VALUES ('{}','{}','{}','{}',{},{},'{}','{}',{},{},'{}');".format(*inp))
    
    cursor.execute(query)  
    
    
    #MongoDB insertion
    #Getting hashtags from the extended tweet if the original tweet is truncated 
    entities =[]
    if record['truncated'] == False:
        text = record['text']
        for i in record['entities']['hashtags']:
            entities.append(i['text'])
    else:
        text = record['extended_tweet']['full_text']
        for i in record['extended_tweet']['entities']['hashtags']:
            entities.append(i['text']) 
    
    try: #If the first 2 words of the tweet is RT but it is not actually retweet
        if record['text'][0:2] =='RT':
            retweet ="Y"
        #Getting hashtags and hashtags from the retweeted object
            rt_entities =[]
            if record['retweeted_status']['truncated'] == False:
                rt_text = record['retweeted_status']['text'] 

                for i in record['retweeted_status']['entities']['hashtags']:
                    rt_entities.append(i['text'])
            else:
                rt_text = record['retweeted_status']['extended_tweet']['full_text']

                for i in record['retweeted_status']['extended_tweet']['entities']['hashtags']:
                     rt_entities.append(i['text'])
        else:
            retweet ="N"
            rt_text = None
            rt_entities = None
    except:
        retweet ="N"
        rt_text = None
        rt_entities = None
    
        
    time = datetime.strftime(datetime.strptime(record['created_at'],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
        
    mydict = {"tweet_id":record['id_str'],"time":time,"user_id":record['user']['id_str'],"handle":record['user']['screen_name'],
              "followers":record['user']['followers_count'],"likes":record['favorite_count'],"hash_orig":entities,"hash_rtwt":rt_entities,"retweet":retweet,"text":text,
             "rtwt_text":rt_text }

    tweets_col.insert_one(mydict)

tweets_col.create_index("followers")

In [37]:
userdb.commit()

query  = "select count(distinct user_id), count(user_id) from user;"
cursor.execute(query)
result = cursor.fetchall()
for x in result:
    print(x)


(7386, 7386)


## Instantiating Redis connection for caching

In [10]:
conn = redis.StrictRedis(host='127.0.0.1', port=6379,charset="utf-8", decode_responses=True)

## Caching data for 4 most famous hashtags

In [24]:
hashtags = ["dogecoin", "doge", "bitcoin", "btc"] 

for m in hashtags:
    myquery = {"$or":[{ "hash_orig": { "$elemMatch": {"$eq":m} } },{ "hash_rtwt": { "$elemMatch": {"$eq":m} } }]}
    mydoc = tweets_col.find(myquery).sort("followers",-1)

    tweets_cnt = tweets_col.count_documents(myquery)
    dist_users = len(tweets_col.distinct('user_id', myquery))

    #ave_query =([{"$group": {"_id":'null', "average": {"$avg":"$followers"} } }])

    p =1
    avg = 0
    for i in mydoc:
        avg = avg+i['followers']
        p+=1

    avg_follow = round(avg/p,0)
    mydoc = tweets_col.find(myquery).sort("followers",-1)
    
    tweet_1 = mydoc[0]['text']
    tweet_2 = mydoc[1]['text']
    
    mydict = {"tweets_cnt":tweets_cnt,"dist_users":dist_users,"avg_follow":avg_follow,"tweet_1":tweet_1,"tweet_2":tweet_2}
    
    key = "hashtag_"+m
    
    conn.hmset(key,mydict)


C:\Users\rajuk\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.


## Caching data for 50 most popular users

In [18]:
query ="select handle from (select handle from user order by followers desc) as b LIMIT 50"
cursor.execute(query)

x = cursor.fetchall()
top_50 = list(map(lambda x:x[0],x))

for i in top_50:

    query ="select user_name,handle,description,followers,statuses,created_date,verified,friends,favourites,protected from user where handle ='{}'".format(i)
    cursor.execute(query)

    x = list(cursor.fetchone())
    x[5] = datetime.strftime(x[5], '%Y-%m-%d')

    myquery = { "handle": {"$eq":i}}
    mydoc = tweets_col.find(myquery).sort("time",-1)
    
    tweet_1 = mydoc[0]['text']

    tweets_cnt = tweets_col.count_documents(myquery)

    mydict = {"user_name":x[0],"description":x[2],"followers":x[3],"statuses":x[4],"created_date":x[5],"verified":x[6],"friends":x[7],"favourites":x[8],"protected":x[9],"tweets_cnt":tweets_cnt,"tweet_1":tweet_1} 

    conn.hmset(x[1],mydict)


C:\Users\rajuk\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.


In [78]:
top_50

['TheStreet',
 'RobMcNealy',
 'OKEx',
 'HuobiGlobal',
 'kennyflorian',
 'DocumentingBTC',
 'DogecoinRise',
 'maxkeiser',
 'BitPay',
 'lopp',
 'RussellOkung',
 'adam3us',
 'Sweepsgg',
 'CryptoBoomNews',
 'jimmysong',
 'JeffBooth',
 'VinnyLingham',
 'PeterMcCormack',
 'BeefEnt',
 'wallstwolverine',
 'JerseyKidPicks',
 'BTC_Archive',
 'bordong2',
 'hentvv',
 'pierre_rochard',
 'statmuse',
 'twobitidiot',
 'stacyherbert',
 'saifedean',
 'GoingParabolic',
 'Pentosh1',
 'CryptoWendyO',
 'lightning',
 'defipulse',
 'nebraskangooner',
 'ThisIsNuse',
 'TheCryptoZombie',
 'HollowManSeries',
 'matt_odell',
 'MustHaveCrypto',
 'CryptoMatrix2',
 'AP4Liberty',
 'parabolictrav',
 'bmurphypointman',
 'LayahHeilpern',
 'MattWallace888',
 'GoldTelegraph_',
 'Bloqport',
 'bitstein',
 'bitcoinagile']

## Creating functions for the search application

In [33]:
def hashtag(z):
    try:
        redis_search = "hashtag_"+z
        
        if conn.exists(redis_search):
            rec = conn.hgetall(redis_search)
            avg_follow=rec['avg_follow']
            dist_users=rec['dist_users']
            tweets_cnt=rec['tweets_cnt']
            tweet_1=rec['tweet_1']
            tweet_2=rec['tweet_2']
            
        else:
            myquery = {"$or":[{ "hash_orig": { "$elemMatch": {"$eq":z} } },{ "hash_rtwt": { "$elemMatch": {"$eq":z} } }]}
            mydoc = tweets_col.find(myquery).sort("followers",-1)

            tweets_cnt = tweets_col.count_documents(myquery)
            dist_users = len(tweets_col.distinct('user_id', myquery))

            #ave_query =([{"$group": {"_id":'null', "average": {"$avg":"$followers"} } }])

            p =1
            avg = 0
            for i in mydoc:
                avg = avg+i['followers']
                p+=1

            avg_follow = round(avg/p,0)
            mydoc = tweets_col.find(myquery).sort("followers",-1)

            tweet_1=mydoc[0]['text']
            tweet_2=mydoc[1]['text']

        text = fontstyle.apply('SUMMARY STATISTICS:', 'bold/white/black_BG')

        summary="""

        Number of tweets: {}

        Number of users who posted the hashtag: {}

        Average no. of followers for the users who tweeted: {}

        """.format(tweets_cnt,dist_users,avg_follow)


        twt_head = fontstyle.apply('Two Tweets from the most followed people ', 'bold/white/black_BG')

        tweet = """"

        1.{}

        2.{}""".format(tweet_1,tweet_2)


        print(text,summary,twt_head,tweet)
    except:
        error = fontstyle.apply('Use alternate search criteria', 'bold/white/black_BG')
        print(error)       

In [18]:
def user(z):
    try:
    ###Implementing cache
        if conn.exists(z):
            x =[None]*10
            rec = conn.hgetall(z)
            x[0] = rec['user_name'] 
            x[5] = rec['created_date'] 
            x[2] = rec['description'] 
            x[6] = rec['verified'] 
            x[9] = rec['protected'] 
            x[3] = rec['followers'] 
            x[7] = rec['friends'] 
            x[4] = rec['statuses'] 
            x[8] = rec['favourites'] 

            twt = rec['tweet_1']
        else:
            query ="select user_name,handle,description,followers,statuses,created_date,verified,friends,favourites,protected from user where handle ='{}'".format(z)
            cursor.execute(query)

            x = list(cursor.fetchone())
            x[5] = datetime.strftime(x[5], '%Y-%m-%d')

            myquery = { "handle": {"$eq":z}} 
            mydoc = tweets_col.find(myquery).sort("time",-1)

            tweets_cnt = tweets_col.count_documents(myquery)



            twt = mydoc[0]['text']

        text = fontstyle.apply('SUMMARY STATISTICS:', 'bold/white/black_BG')    
        summary="""

        User Name: '{}'

        Created date : {}

        Description : '{}'

        Verified : '{}'

        Protected : '{}'

        Followers : {}

        Friends : {}

        Statuses : {}

        Favourites : {}

        """.format(x[0],x[5],x[2],x[6],x[9],x[3],x[7],x[4],x[8])


        twt_head = fontstyle.apply('Latest Tweet from the User:', 'bold/white/black_BG')


        tweet = """"

        1.{}

        """.format(twt)


        print(text,summary,twt_head,tweet)
    except:
        error = fontstyle.apply('Use alternate search criteria', 'bold/white/black_BG')
        print(error)

In [7]:
def time_range(minm, maxm):
    try:
        minm = str(minm)
        maxm = str(maxm)
        myquery = {"time":{"$gte":minm,"$lt":maxm}}
        mydoc = tweets_col.find(myquery).sort("followers",-1)

        tweets_cnt = tweets_col.count_documents(myquery)
        dist_users = len(tweets_col.distinct('user_id', myquery))

        #ave_query =([{"$group": {"_id":'null', "average": {"$avg":"$followers"} } }])

        p =1
        avg = 0
        for i in mydoc:
            avg = avg+i['followers']
            p+=1

        avg_follow = round(avg/p,0)
        mydoc = tweets_col.find(myquery).sort("followers",-1)

        text = fontstyle.apply('SUMMARY STATISTICS:', 'bold/white/black_BG')

        summary="""
        
        Number of tweets: {}

        Number of users who posted within the time range : {}

        Average no. of followers for the users who tweeted: {}

        """.format(tweets_cnt,dist_users,avg_follow)


        twt_head = fontstyle.apply('2 Tweets from the most followed people ', 'bold/white/black_BG')

        tweet = """"

        1.{}

        2.{}""".format(mydoc[0]['text'],mydoc[1]['text'])


        print(text,summary,twt_head,tweet)
    except:
        error = fontstyle.apply('Use alternate search criteria', 'bold/white/black_BG')
        print(error)        


In [8]:
def word(z):
    try:
        myquery = {"$or":[{ "text": {"$regex":z} } ,{ "rtwt_text": {"$regex":z} } ]}
        mydoc = tweets_col.find(myquery).sort("followers",-1)


        tweets_cnt = tweets_col.count_documents(myquery)
        dist_users = len(tweets_col.distinct('user_id', myquery))

        #ave_query =([{"$group": {"_id":'null', "average": {"$avg":"$followers"} } }])

        p =1
        avg = 0
        for i in mydoc:
            avg = avg+i['followers']
            p+=1

        avg_follow = round(avg/p,0)
        mydoc = tweets_col.find(myquery).sort("followers",-1)

        text = fontstyle.apply('SUMMARY STATISTICS:', 'bold/white/black_BG')

        summary="""
        
        Number of tweets: {}

        Number of users who tweeted the selected word : {}

        Average no. of followers for the users who tweeted: {}

        """.format(tweets_cnt,dist_users,avg_follow)


        twt_head = fontstyle.apply('Two Tweets from the most followed people ', 'bold/white/black_BG')

        tweet = """"

        1.{}

        2.{}""".format(mydoc[0]['text'],mydoc[1]['text'])


        print(text,summary,twt_head,tweet)
    except:
        error = fontstyle.apply('Use alternate search criteria', 'bold/white/black_BG')
        print(error)


## Search application

In [49]:
sig ="Y"
while sig=="Y":
    print("The attribute you want to search on: hashtag, user, date or word?")
    x =input()
    if x=='hashtag':
        clear_output()
        print("what is the hashtag?")
        has = input()
        clear_output()
        hashtag(has)
    if x=='date':
        clear_output()
        print("Enter the from date and to date seperated by a comma in the foramt Y-m-d H:M:S ex: 2021-04-26 14:12:17,2021-04-26 14:12:19 ")
        minm, maxm = input().split(",")
        clear_output()
        time_range(minm,maxm)
    if x=='user':
        clear_output()
        print("what is the user handle?")
        q = input()
        clear_output()
        user(q)
    if x=='word':
        clear_output()
        print("what is the word?")
        p = input()
        clear_output()
        word(p)
    print("Do we want to restart the application (Y/N?)")
    sig = input()
    clear_output()
print("Adios!")

Adios!


## Performance - User search

In [50]:
start_time = time.time()
user('TheStreet')
print("--- %s seconds ---" % (time.time() - start_time))


SUMMARY STATISTICS: 

        User Name: 'TheStreet'

        Created date : 2008-06-30

        Description : 'TheStreet brings you the best business news and premium investing ideas and stock analysis around. We want to help you make money, lots of it.'

        Verified : 'True'

        Protected : 'False'

        Followers : 777506

        Friends : 1124

        Statuses : 203269

        Favourites : 10351

         Latest Tweet from the User: "

        1.A tight end for the Kansas City Chiefs will take his 2021 salary in #bitcoin. https://t.co/QqnFM0HxcA

        
--- 0.000997304916381836 seconds ---


In [51]:
start_time = time.time()
user('ankon99')
print("--- %s seconds ---" % (time.time() - start_time))

SUMMARY STATISTICS: 

        User Name: 'Ankon'

        Created date : 2021-04-22

        Description : 'Student'

        Verified : 'False'

        Protected : 'False'

        Followers : 1

        Friends : 46

        Statuses : 43

        Favourites : 16

         Latest Tweet from the User: "

        1.RT @Phemex_official: We are giving away $DOGE again! Help our Twitter account to reach 200k followers - a prize pool of up to $40k is waiti…

        
--- 0.02665090560913086 seconds ---


## Performance - Hashtag search

In [34]:
start_time = time.time()
hashtag('doge')
print("--- %s seconds ---" % (time.time() - start_time))

SUMMARY STATISTICS: 

        Number of tweets: 537

        Number of users who posted the hashtag: 397

        Average no. of followers for the users who tweeted: 1640.0

         Two Tweets from the most followed people  "

        1.RT @suryocokroo: @latokens When combined into one
 #DogeOnLatoken strength, of course, it 
has #strong #market #technology
So!
there is no…

        2.#atlanta #atl #georgia #crypto #cryptocurrency #cryptocurrenices #dogearmy #dogecoin #affiliatemarketing PipeFlare - free #ZEC and #doge #faucet - free games and earn #cryptocoin https://t.co/7tvVbbecvT
--- 0.0009629726409912109 seconds ---


In [35]:
start_time = time.time()
hashtag('crypto')
print("--- %s seconds ---" % (time.time() - start_time))

SUMMARY STATISTICS: 

        Number of tweets: 964

        Number of users who posted the hashtag: 842

        Average no. of followers for the users who tweeted: 2458.0

         Two Tweets from the most followed people  "

        1.RT @HuobiSg: Comment below to cast your vote now! 
#crypto #Huobi #Bitcoin #Oscars https://t.co/D6gFAEM8fm

        2.RT @suryocokroo: @latokens When combined into one
 #DogeOnLatoken strength, of course, it 
has #strong #market #technology
So!
there is no…
--- 0.05269312858581543 seconds ---
